In [1]:
import time
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys

sys.path.insert(0,"../")

from app.mlcode.predict_price_movements import BollingerBandsPredictor
from app.mlcode.determine_trading_state import DetermineTradingState
from app.mlcode.utils import read_in_data, read_in_yaml, running_on_aws, setup_logging, update_yaml_config

%pylab inline

import logging

from tqdm.notebook import tqdm
tqdm.pandas()

Importing plotly failed. Interactive plots will not work.


Populating the interactive namespace from numpy and matplotlib


/Users/jonathanhilgart/Library/Caches/pypoetry/virtualenvs/go-trader-aTWd7ZB6-py3.8/lib/python3.8/site-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['datetime']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


## Setup data
- This is all taken from /tmp which should be downloaded first using make

`make download_configs_and_data`

In [2]:
base_path = "configs/"
coin = "btc"
coin_to_predict = coin +"_"

constants = read_in_yaml( base_path + "constants.yml", running_on_aws())
    
trading_state_filename = base_path + coin_to_predict + "trading_state_config.yml"
trading_constants = read_in_yaml(trading_state_filename, running_on_aws())

won_lost_amount_filename = base_path + coin_to_predict + "won_and_lost_config.yml"
won_and_lost_amount_constants = read_in_yaml(won_lost_amount_filename, running_on_aws())
    

actions_to_take_filename =  base_path + coin_to_predict + "actions_to_take.yml"
actions_to_take_constants = read_in_yaml(actions_to_take_filename, running_on_aws())


ml_constants = read_in_yaml("../tmp/" +constants["ml_config_filename"].split("/")[1], running_on_aws())

2022-03-30 21:55:26,786: : read_in_yaml  Reading in configs/constants.yml
2022-03-30 21:55:26,805: : read_in_yaml  Key = btc_product_code Value = BTC/USD
2022-03-30 21:55:26,806: : read_in_yaml  Key = tcn_modelname_btc Value = tcn_btc
2022-03-30 21:55:26,806: : read_in_yaml  Key = nbeats_modelname_btc Value = nbeats_btc
2022-03-30 21:55:26,808: : read_in_yaml  Key = bitcoin_csv_filename Value = tmp/historic_crypto_prices - bitcoin_jan_2017_sep_4_2021 copy.csv
2022-03-30 21:55:26,811: : read_in_yaml  Key = tcn_filename_btc Value = checkpoint_4499.pth.tar
2022-03-30 21:55:26,812: : read_in_yaml  Key = nbeats_filename_btc Value = checkpoint_257.pth.tar
2022-03-30 21:55:26,813: : read_in_yaml  Key = eth_product_code Value = ETH/USD
2022-03-30 21:55:26,815: : read_in_yaml  Key = tcn_modelname_eth Value = tcn_eth
2022-03-30 21:55:26,816: : read_in_yaml  Key = nbeats_modelname_eth Value = nbeats_eth
2022-03-30 21:55:26,818: : read_in_yaml  Key = etherum_csv_filename Value = tmp/historic_crypt

In [3]:
bitcoin_df = read_in_data("../tmp/" + constants["bitcoin_csv_filename"].split("/")[1], running_on_aws())
etherum_df = read_in_data("../tmp/" +constants["etherum_csv_filename"].split("/")[1], running_on_aws())
sol_df = read_in_data("../tmp/" +constants["sol_csv_filename"].split("/")[1], running_on_aws())
matic_df = read_in_data("../tmp/" +constants["matic_csv_filename"].split("/")[1], running_on_aws())
link_df = read_in_data("../tmp/" +constants["link_csv_filename"].split("/")[1], running_on_aws())

2022-03-30 21:55:26,970: : read_in_data  Input file ../tmp/historic_crypto_prices - bitcoin_jan_2017_sep_4_2021 copy.csv
2022-03-30 21:55:26,989: : read_in_data                 open     high      low    close       volume
date                                                       
2017-01-01   963.66  1003.08   958.70   998.33  147775008.0
2017-01-02   998.62  1031.39   996.70  1021.75  222184992.0
2017-01-03  1021.60  1044.08  1021.60  1043.84  185168000.0
2017-01-04  1044.40  1159.42  1044.40  1154.73  344945984.0
2017-01-05  1156.73  1191.10   910.42  1013.38  510199008.0
2022-03-30 21:55:27,002: : read_in_data                 open     high      low    close        volume
date                                                        
2022-03-26  44331.0  44792.0  44101.0  44531.0  2.278341e+07
2022-03-27  44531.0  46962.0  44448.0  46842.0  5.399869e+07
2022-03-28  46842.0  48220.0  46661.0  47160.0  8.354505e+07
2022-03-29  47160.0  48185.0  47001.0  47456.0  8.208843e+07
2022-03-30 

2022-03-30 21:55:27,132: : read_in_yaml  Reading in ../tmp/ml_config.yml
2022-03-30 21:55:27,141: : read_in_yaml  Key = prediction_params Value = {'bollinger_window': 5, 'no_of_std': 1.25, 'lookback_window': [15, 30, 45], 'prediction_n_days': 7, 'model_name': ['TCN', 'NBEATS'], 'work_dir': '/mnt/shared-storage'}
2022-03-30 21:55:27,142: : read_in_yaml  Key = hyperparameters_tcn Value = {'dropout': 0.1, 'random_state': 0, 'dilation_base': 2, 'kernel_size': 14, 'num_filters': 7, 'num_layers': 7, 'weight_norm': True, 'epochs': 60}
2022-03-30 21:55:27,143: : read_in_yaml  Key = hyperparameters_nbeats Value = {'random_state': 0, 'num_blocks': 5, 'layer_widths': 256, 'epochs': 11}
2022-03-30 21:55:27,144: : read_in_yaml  ----------


 - BTC

In [5]:
predictor = BollingerBandsPredictor(
    coin, constants, ml_constants, bitcoin_df, additional_dfs=[etherum_df]  # spy_df
)

In [6]:
price_prediction = predictor.predict()

2022-03-30 21:55:27,166: : predict  Slicing dataframes
2022-03-30 21:55:27,168: : _slice_df  Slice date , earliest day of data for main df, = 2017-01-01 00:00:00
2022-03-30 21:55:27,170: : predict  Building Bollinger Bands
2022-03-30 21:55:27,181: : _build_technical_indicators  ---- Adding Bollinger Bands ----
2022-03-30 21:55:27,182: : _build_technical_indicators                 open     high      low    close        volume  Rolling Mean  \
date                                                                         
2022-03-26  44331.0  44792.0  44101.0  44531.0  2.278341e+07       43628.8   
2022-03-27  44531.0  46962.0  44448.0  46842.0  5.399869e+07       44521.8   
2022-03-28  46842.0  48220.0  46661.0  47160.0  8.354505e+07       45375.2   
2022-03-29  47160.0  48185.0  47001.0  47456.0  8.208843e+07       46064.0   
2022-03-30  47455.0  47716.0  46329.0  47085.0  5.690887e+07       46614.8   

            Bollinger High  Bollinger Low  
date                                     

  0%|          | 0/11 [00:00<?, ?it/s]

2022-03-30 21:55:27,749: : _init_model  Time series values are 64-bits; casting model to float64.


  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

...

  0%|          | 0/11 [00:00<?, ?it/s]

2022-03-30 22:04:25,245: : _train_models  Have thread = [<darts.models.forecasting.nbeats.NBEATSModel object at 0x105a8a3a0>, <darts.models.forecasting.tcn_model.TCNModel object at 0x105a8a670>]_tcn
2022-03-30 22:04:25,267: : _train_models  Have thread = [<darts.models.forecasting.nbeats.NBEATSModel object at 0x105a8a6d0>, <darts.models.forecasting.tcn_model.TCNModel object at 0x127fad130>]_nbeats


2022-03-30 22:05:29,873: : _train_models  Have thread = [<darts.models.forecasting.nbeats.NBEATSModel object at 0x105a8a6d0>, <darts.models.forecasting.tcn_model.TCNModel object at 0x127fad130>]_tcn
2022-03-30 22:05:29,876: : _train_models  Have thread = [<darts.models.forecasting.nbeats.NBEATSModel object at 0x127fad880>, <darts.models.forecasting.tcn_model.TCNModel object at 0x127fadbe0>]_nbeats


2022-03-30 22:06:10,993: : _train_models  Have thread = [<darts.models.forecasting.nbeats.NBEATSModel object at 0x127fad880>, <darts.models.forecasting.tcn_model.TCNModel object at 0x127fadbe0>]_tcn
2022-03-30 22:06:10,997: : predict  making predictions


2022-03-30 22:06:11,792: : _make_prediction   Model = nbeats_btc_lookback_15 Lookback = 7 Prediction = 48789.55535113841
2022-03-30 22:06:11,839: : _make_prediction   Model = tcn_btc_lookback_15 Lookback = 7 Prediction = 44772.65043697051
2022-03-30 22:06:12,100: : _make_prediction   Model = nbeats_btc_lookback_30 Lookback = 7 Prediction = 44028.93059533704
2022-03-30 22:06:12,118: : _make_prediction   Model = tcn_btc_lookback_30 Lookback = 7 Prediction = 45408.1201197201
2022-03-30 22:06:12,209: : _make_prediction   Model = nbeats_btc_lookback_45 Lookback = 7 Prediction = 43325.42615116384
2022-03-30 22:06:12,225: : _make_prediction   Model = tcn_btc_lookback_45 Lookback = 7 Prediction = 44891.21608323259
2022-03-30 22:06:12,227: : predict  prediction = 45202.64978959374
